# Analysis of vacancy and foreclosures, transactions, sheriff's auctions, and armslength sales

In [16]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pandas as pd
import numpy as np
import os

%matplotlib inline

path = '/'.join(os.getcwd().split('/')[:-1])
print(path)

/Volumes/Dropbox/largetransfer/luc/carter


In [46]:
tci = pd.read_csv(path+'/data/model_data/tci_1_0.csv', index_col = 0,parse_dates=['Date'])
tci = tci[['parcel','vacant','SPA_NAME','Date']]
# tci = pd.read_csv(path+'/data/model_data/tci_2_2.csv', parse_dates=['Date'], dtype={'Parcel ID':str})
# tci = pd.merge(tci, df[['NAME10','SPA_NAME']], how='left', right_index=True, left_on='parcel')
ppns = set(tci['parcel'])
dates = dict(zip(tci.parcel, tci.Date))
print(tci.shape)

(113132, 4)


## Foreclosures

In [18]:
fc = pd.read_csv(path+'/data/clean_data/foreclosure_filings2.csv', parse_dates=[2])
fc = fc.sort_values('filedate').groupby('caseno').last()
fc = fc.rename(columns={'filedate':'date'}).reset_index()

fc.columns

Index([u'caseno', u'parcel', u'status', u'date', u'defendant',
       u'parcel_address', u'dateid', u'case_title', u'plaintiff', u'FORE',
       u'LATITUDE', u'LONGITUDE', u'zip_fore'],
      dtype='object')

In [19]:
fc_parcel = set(fc.parcel)
tci['fc'] = tci.apply(lambda x: x.parcel in fc_parcel, axis=1)
pd.crosstab(tci.fc, tci.vacant)

vacant,0,1
fc,,
False,79154,3836
True,23940,6202


In [20]:
sum(tci.vacant), sum(tci.fc)

(10038, 30142)

#### Has it been foreclosed before?

So of the 30,142 parcels that have been foreclosed between 1/3/2006 and 11/6/2015, 6,202 are currently vacant, of the 10,038 total residential vacancies as measured by TCI. This means that 3,836 parcels were determined to be vacancy but never have been foreclosed. 

In [21]:
fc_parcel = set(fc[fc.status=='Inactive'].parcel)
tci['fc'] = tci.apply(lambda x: x.parcel in fc_parcel, axis=1)
pd.crosstab(tci.fc, tci.vacant)

vacant,0,1
fc,,
False,79967,4135
True,23127,5903


In [22]:
fc_parcel = set(fc[fc.status=='Active'].parcel)
tci['fc'] = tci.apply(lambda x: x.parcel in fc_parcel, axis=1)
pd.crosstab(tci.fc, tci.vacant)

vacant,0,1
fc,,
False,101689,9455
True,1405,583


In [23]:
tem = pd.merge(fc[['parcel','status']].groupby('parcel').count(), tci[['parcel','vacant']], how='right', left_index=True, right_on='parcel').fillna(0)
tem = tem[['status','vacant']].groupby('status').agg([sum,len])
tem['percent'] = tem.vacant['sum']/tem.vacant['len']
tem.T

status                  0             1            2            3           4  \
vacant  sum   3836.000000   4193.000000  1579.000000   330.000000   81.000000   
        len  82990.000000  22045.000000  6255.000000  1422.000000  326.000000   
percent          0.046222      0.190202     0.252438     0.232068    0.248466   

status               5          6      7  
vacant  sum  14.000000   4.000000  1.000  
        len  71.000000  15.000000  8.000  
percent       0.197183   0.266667  0.125

## Sheriff's Auctions

In [24]:
sa = pd.read_csv(path+'/data/clean_data/sheriff_auction.csv', parse_dates=[2])#, encoding="ISO-8859-1")

In [25]:
reo = pd.read_csv(path+'/data/extra_data/reo.csv', encoding="utf-8")
reo = reo.groupby('GRANTEE1').last()

In [26]:
purchaser = sa.groupby('scaseno').last()
purchaser = purchaser.groupby('purchaser').count().sort_values('parcel',ascending=False)[['parcel']]


In [27]:
# purchaser = sa.groupby('scaseno').last().groupby('purchaser').count().sort_values('parcel',ascending=False)[['parcel']]
purchaser = purchaser.rename(columns={'parcel':'purchaser_count'})
sa = pd.merge(sa, purchaser, how='left', left_on='purchaser',right_index=True)
# sa = sa.sort_values('salesdt').groupby('parcel').last()
sa = sa.rename(columns={'salesdt':'date'})
# sa = sa[['date','plaintiff_count']]
# sa['type'] = 'sa'
# sa = sa.reset_index()
sa['purchaser_count'] = sa['purchaser_count'].fillna(0)
sa.columns

Index([u'parcel', u'address', u'date', u'saleno', u'sold_amt', u'appraisal',
       u'minbid', u'scaseno', u'withdrawn', u'plaintiff', u'sdefend',
       u'purchaser', u'paddress', u'rparty', u'raddress', u'attorney',
       u'descript', u'location', u'defend_att', u'with', u'status', u'ssold',
       u'SHF', u'LATITUDE', u'LONGITUDE', u'purchaser_count'],
      dtype='object')

In [ ]:
# al = pd.read_csv(path+'/data/clean_data/armslength.csv',dtype=str)
# al = al.drop_duplicates()

# months = dict(zip(['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'],range(1,13)))

# def parse_date(x):
#     day = int(x[0:2])
#     month = months[x[2:5]]
#     year = int(x[5:])
#     return dt.datetime(year,month,day)

# al['date'] = al.mdate.apply(parse_date)
# # al = al[al.date < dt.datetime(2015,6,1)]

# al = al.rename(columns={'PROPERTY_NUMBER':'parcel'})
# al = al[['date','parcel']]
# al['type'] = 'al'
# al['plaintiff_count'] = 0

In [ ]:
# t2 = pd.read_csv(path+'/data/clean_data/transfers.csv', parse_dates=['mdate'], dtype=str)

In [79]:
t = pd.read_csv(path+'/data/clean_data/transfers.csv', parse_dates=['mdate'], dtype=str)

t = t.rename(columns={'PROPERTY_NUMBER':'parcel','mdate':'date'})
t['type'] = 't'
t['REO'] = 0

In [29]:
t = pd.merge(t, reo, how='left',left_on='GRANTEE1', right_index=True)

In [87]:
t.loc[(t.DEED_TYPE.isin(['Sheriffs Deed',
 'Sheriffs Deed Ex'])),'shf'] = 1

In [31]:
reo

,stdgr1,finalgr1
GRANTEE1,,
#NAME?,CENTEX HOME EQUITY CORP,FINANCIAL INSTITUTION
AAMES CAPITAL CORP,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES CAPITAL CORP. & AAMES FUNDING CORPORATION,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES CAPITAL CORPORATION,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES FINANCIAL CORPORATION,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES FUNDING CORPORATION,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES FUNDING CORPORATION DBA AAMES HOME LOAN,AAMES CAPITAL CORP,FINANCIAL INSTITUTION
"AAMES FUNDING CORPORATION, DBA AAMES HOME LOAN",AAMES CAPITAL CORP,FINANCIAL INSTITUTION
AAMES HOME LOAN,AAMES CAPITAL CORP,FINANCIAL INSTITUTION


In [ ]:
temp = tci[tci.parcel.isin(t[(t.REO==1)&(t.date > dt.datetime(2013,1,1))].parcel)]
sum(temp.vacant), len(temp)

In [32]:
fc2 = fc[['date','parcel']]
fc2['type'] = 'fc'
fc2['REO'] = 0

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [33]:
sa2 = sa[['date','parcel','sold_amt']]
sa2['REO'] = 0
sa2['type'] = 'sa'

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [95]:
df = fc2.append(sa2.loc[sa2.sold_amt.notnull(),['date','REO','parcel','type']]).append(t[['date','parcel','type','REO','DEED_TYPE','GRANTEE1']])
df = df.sort_values('date')
df = df.loc[df.apply(lambda x: x.date < dates[x.parcel], axis=1)]

In [100]:
last = df.groupby('parcel').last()
sub = last[(last['type'] == 't') & (last['DEED_TYPE'].isin(['Sheriffs Deed',
 'Sheriffs Deed Ex'])) & (last['REO'])]#&(last['plaintiff_count']>-1)]

In [101]:
test = set(sub.index)
sum(tci[tci.parcel.isin(test)].vacant), len(tci[tci.parcel.isin(test)].vacant)

(546, 1203)

In [107]:
tci.loc[tci.parcel.isin(test),['SPA_NAME','vacant']].groupby('SPA_NAME').agg([sum,len])

vacant     
                           sum  len
SPA_NAME                           
Bellaire-Puritas            17   46
Broadway-Slavic Village     43   73
Brooklyn Centre             11   27
Buckeye-Shaker Square       18   43
Buckeye-Woodhill            10   18
Central                      4   10
Clark-Fulton                 6   22
Collinwood-Nottingham       28   58
Cudell                      16   32
Detroit Shoreway             7   23
Edgewater                    2    7
Euclid-Green                 7   19
Fairfax                      8   14
Glenville                   65   95
Goodrich-Kirtland Pk         6    8
Hough                        7   18
Jefferson                   28   68
Kamm's                      20   53
Kinsman                     12   20
Lee-Harvard                 26   63
Lee-Seville                  8   23
Mount Pleasant              44   84
North Shore Collinwood      24   55
Ohio City                    3    8
Old Brooklyn                36  102
St.Clair-Superior            6   13
Stockyards                  14   31
Tremont                      2    9
Union-Miles                 37   86
University                   1    3
West Boulevard              30   72

In [66]:
import re

def find_REO(s):
    if isinstance(s,str) or isinstance(s,unicode):
        s = s.lower()
        if re.search("llc", s):
            return True
        if re.search("bank", s):
            return True
        if re.search("mortg", s):
            return True
        if re.search("mort.", s):
            return True
        if re.search("comp", s):
            return True
        if re.search("corp", s):
            return True
        if re.search("fannie", s):
            return True
        if re.search("housing", s):
            return True
        if re.search("sec.", s):
            return True
        if re.search("loan", s):
            return True
        return False
    else:
        return False

In [70]:
sum(sa['REO'])

22976

In [5]:
sa.groupby('purchaser').count().sort_values('date',ascending=False)

NameError: name 'sa' is not defined

In [68]:
sa['REO'] = sa.purchaser.apply(find_REO)

In [88]:
t['REO'] = t.GRANTEE1.apply(find_REO)

In [91]:
t[(t.shf==1)&t.REO]

,TRANSFER_TO_ID,WHS_ID,TRANSFER_HISTORY_ID,TRANSFER_ORDER,PARCEL_ID,parcel,GRANTEE1,GRANTOR1,date,SALES_AMOUNT,...,GRANTOR_IS_RELATIVE,TRADE,TRANSFER_TYPE_DESCR,SALES_SOURCE_DESCR,EXEMPTCODE_DESCR,TRANSFER_FROM_PARCEL,UPDATE_DATE,type,REO,shf
102,591491638,4282084,676448184,4,102024,001-02-024,BANK OF AMERICA NA,"Waszak,Helena V",2014-09-24,228000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
157,1687613006,4286307,972164435,15,104007,001-04-007,JPMORGAN CHASE BANK NATIONAL ASSOCIATION,"Sikora, Peter",2014-04-11,910000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
169,2040389458,4286316,871525777,11,104016,001-04-016,"GMAC MORTGAGE, LLC","lange, gerald d.",2009-09-16,290000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
230,849225706,4288480,838247536,3,105025,001-05-025,THE BANK OF NEW YORK MELLON,"Lebovitz, Sanford",2013-02-26,150000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
270,1831971867,4290658,814858079,8,106025,001-06-025,"DEUTSCHE BANK TRUST CO. AMERICAS, TRUSTEE","CHAPPELL, RENEE, & ORLANDO",2011-08-17,150000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
309,1088011691,4292833,249970982,4,107018,001-07-018,"WELLS FARGO BANK, N.A. AS TRUSTEE","Kerfonta, Christina R.",2007-11-16,30000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
313,9673909,4292835,1558894809,10,107020,001-07-020,US BANK N.A. TRUSTEE,"IGLESSIAS, JACOB",2009-06-11,20000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
322,938538843,4292845,728023415,9,107030,001-07-030,KEYBANK NATIONAL ASSOCIATION,Dearmond Margaret A.,2005-09-01,35000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
324,1442093915,4292845,926113198,11,107030,001-07-030,US BANK NA AS TRUSTEE,"PALLANT, BRAD",2011-05-04,23334.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1
327,1499158716,4292846,1636589576,13,107031,001-07-031,GMAC MORTGAGE CORP.,"Drucilla, Ivy",2004-01-09,44000.0,...,0.0,0.0,Change Ownership,NaN,NaN,NaN,07NOV2014,t,True,1


In [83]:
t.shape

(210880, 62)

In [128]:
len(tci[tci.parcel.isin(df[(df.date>dt.datetime(2015,1,1)) & (df.DEED_TYPE.isin(['Sheriffs Deed',
 'Sheriffs Deed Ex']))].parcel)].vacant)

498

In [118]:
df.groupby('parcel').last().groupby('type').()

,DEED_TYPE,GRANTEE1,REO,date
type,,,,
fc,7635,7635,11337,11337
sa,754,754,1074,1074
t,68664,68664,68664,68664


In [82]:
sum(t.REO)

48993

In [103]:
find_REO(sa.purchaser.iloc[6])

True

In [107]:
s = sa.groupby('scaseno').last()

In [113]:
ts = tci[tci.parcel.isin(s[(s.REO) & (s.date > dt.datetime(2011,1,1))].parcel)]
sum(ts.vacant), len(ts)

(1289, 5716)

In [ ]:
spas_sa = tci[tci.parcel.isin(test)][['SPA_NAME','vacant']].groupby('SPA_NAME').agg([sum,len])
spas_sa['percent'] = spas_sa.vacant['sum']*1.0/spas_sa.vacant['len']
print(spas_sa.sort_values('percent'))

In [ ]:
for i in spas_sa.index:
    a = sa[sa.parcel.isin(test.intersection(tci[tci.SPA_NAME==i].parcel))]
    spas_sa.loc[spas_sa.index==i,'median_sa'] = a.sort_values('date')['date'].iloc[round(len(a)/2)]
#     print(i,a.sort_values('date')['date'].iloc[round(len(a)/2)])
#     print(np.median(sa[sa.parcel.isin(test.intersection(tci[tci.SPA_NAME==i].parcel))].date))

In [ ]:
spas_sa

In [ ]:
tci[tci.parcel.isin(set(fc.parcel))][['SPA_NAME','vacant']].groupby('SPA_NAME').agg([sum,len])

In [ ]:
len(tci.loc[tci.vacant==1,'parcel'][tci.loc[tci.vacant==1,'parcel'].isin(set(sa.parcel))])

In [ ]:
sum(tci['vacant']), len(tci)

In [ ]:
sum(tci[tci.parcel.isin(sub)].vacant), len(tci[tci.parcel.isin(sub)].vacant)

In [ ]:
last = df.groupby('parcel').last()
sub = set(last[last['type'] == 'sa'].index)

In [ ]:
second_last = df.groupby('parcel').nth(-2)
sub2 = set(second_last[second_last['type']=='fc'].index)

In [ ]:
len(sub.intersection(sub2)), len(sub), len(sub2)

In [ ]:
fc_to_sa = sub.intersection(sub2)

In [ ]:
sum(tci[tci.parcel.isin(fc_to_sa)].vacant), len(tci[tci.parcel.isin(fc_to_sa)].vacant)

In [ ]:
df[df.parcel=='137-25-057']

In [ ]:
a = second_last[second_last.index.isin(sub)]

In [ ]:
pitt_vacant = set(second_last[second_last.index.isin(sub)].index)

In [ ]:
pv = pd.read_csv(path+'/data/clean_data/postal_vacancy.csv', parse_dates=3, index_col=0)
pv.date = pv.date.apply(lambda x: dt.datetime(int(x[0:4]),int(x[5:7]),int(x[8:10])))
pv = pv.sort_values('date',ascending=False)
pv = pv.loc[pv.apply(lambda x: x.date < dates[x.PARCEL], axis=1)]

In [ ]:
pv = pv[pv.vindall=='Y']
pv_copy = pv.copy().groupby('PARCEL').count().reset_index()

pv_copy['pv_count'] = pv_copy.apply(lambda x: \
        len(pv[(pv.PARCEL == x.PARCEL) & (pv.date > dates[x.PARCEL]-pd.DateOffset(years=2))]), axis=1)

In [ ]:
pv2 = pv.groupby('PARCEL').last()

In [ ]:
tci[tci.parcel.isin(pv2[pv2.vindall=='Y'].index)].groupby('vacant').count()

In [ ]:
sns.distplot(df2[df2.vacant==1].date,bins=np.linspace(0,30,30));
sns.distplot(df2[df2.vacant==0].date,bins=np.linspace(0,30,30));

In [ ]:
df2 = pd.merge(df.groupby('parcel').last(), tci[['vacant','ppn']], how='left',left_index=True, right_on='ppn')

In [ ]:
df2[['type','vacant']].groupby('type').agg([sum,len])

In [ ]:
d = df2[df2['type']=='fc']
d['year'] = d.date.apply(lambda x: x.year)

In [ ]:
d[['year','vacant']].groupby('year').agg([sum,len])

In [ ]:
plt.plot(fc.set_index('date').groupby(['type']).resample('m', 'count'))

In [ ]:
t = tci[tci.ppn.isin(fc[fc.date > dt.datetime(2015,3,1)].parcel)].vacant

In [ ]:
sum(t),len(t)

In [ ]:
fc['year'] = fc['date']